In [4]:
import pandas as pd
import numpy as np
import bqplot
import yt

Symbol_T = 'Time'
Symbol_S = 'Total Sightings'
Symbol_D = 'Total Duration'

In [5]:
from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)
from traitlets import link

from ipywidgets import ToggleButtons, VBox, HTML, Dropdown, HBox

In [6]:
names = ["date", "city", "state", "country", "shape", "duration_seconds",
         "duration_reported", "description", "report_date", "latitude",
         "longitude"]
fn = "/srv/nbgrader/data/ufo-scrubbed-geocoded-time-standardized.csv"
ufo = pd.read_csv(fn, names = names, parse_dates = ["date", "report_date"])

FileNotFoundError: File b'/srv/nbgrader/data/ufo-scrubbed-geocoded-time-standardized.csv' does not exist

In [ ]:
ufo['year'] = [i.year for i in ufo['date']]
ufo.set_index('year')

In [ ]:
total_sightings = ufo.groupby(['state','year'])['report_date'].count()
total_time = ufo.groupby(['state','year'])['duration_seconds'].sum()

In [ ]:
total_sightings_year = ufo.groupby('year')
total_sightings_year = total_sightings_year['date'].count()
total_duration_year = ufo.groupby('year')['duration_seconds'].sum()

### Define a Figure

In [ ]:
total_S = total_sightings['tx']
total_D = total_time['tx']

In [ ]:
#total_S.values()
total_S.index

In [ ]:
import bqplot.pyplot as bqp

In [ ]:
dt_x_fast = DateScale()#min = min(ufo['date']), max = max(ufo['date']))
lin_y = LinearScale()#min = 0, max = 10000)

x_ax = Axis(label = 'Year', scale = dt_x_fast)
y_ay_S = Axis(label = (Symbol_S + ' of the selected state'),
           scale = lin_y, orientation = 'vertical')
y_ay_D = Axis(label = (Symbol_D + ' of the selected state'),
           scale = lin_y, orientation = 'vertical')
lc = Lines(x = total_S.index.values, y = total_S[:], colors = ['orange'],scales={'x':dt_x_fast,'y':lin_y})
#lc2 = Lines(x = list(total_D.keys()), y = list(total_D.values()),colors = ['blue'],scales={'x':dt_x_fast,'y':lin_y})

### Define the type of selector we would like

In [ ]:
intsel_fast = FastIntervalSelector(scale = dt_x_fast, marks = [lc,lc2])

### Define a function that will be called when the FastIntervalSelector is interacted with

In [ ]:
md = {}
def fast_interval_change_callback(change):
    ind = pd.to_datetime(change.new)
    tot = ufo.loc[ind[0]:ind[1]]["Time"].sum()
    db_fast.value = 'The selected time period is '+ str(change.new) + 'total:' + str(tot)

### Connect the selectors to the function


In [ ]:
intsel_fast.observe(fast_interval_change_callback,names = ['selected'])

### Use the HTML widget to see the value of what we are selecting and modify it when an interaction is performed on the selector

In [ ]:
db_fast = HTML()
db_fast.value = 'The selected time period is ' + str(intsel_fast.selected)


fig_fast_insel_sights = Figure(marks = [lc], axes = [x_ax, y_ay_S],
                               interaction = intsel_fast)
fig_fast_insel_time = Figure(marks = [lc2], axes = [x_ax, y_ay_D],
                            interaction = intsel_fast)

def watch_dropdown(change):
    db_fast.value = str((change.old, change.new))
new_dropdown = Dropdown(options = ['Total Sightings','Total Time'])
new_dropdown.observe(watch_dropdown, names = ['value'])
VBox([db_fast,fig_fast_insel_sights, new_dropdown])


In [ ]:
import us

In [ ]:
cap_select = Dropdown(options=['Total Sightings','Total Duration Time'],description='Caption:')
def onCaptionSelected(change):
    cap = change['new']
    if cap == 'Total Sightings':
        states_map.color = fips_count.to_dict()
    if cap == 'Total Duration Time':
        states_map.color = total_time_all.to_dict()
        
cap_select.observe(onCaptionSelected,'value') 

In [ ]:
abbr_to_fits = us.states.mapping('abbr', 'fips')
ufo["fips"] = ufo["state"].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))
fips_count = ufo.groupby("fips")["duration_seconds"].count()
total_time_all = ufo.groupby("fips")["duration_seconds"].sum()

map_styles = {'scales': {'projection': bqplot.AlbersUSA(),
                         'color': bqplot.ColorScale(colors=["red","yellow", "blue"])},
              'color': fips_count.to_dict()}
map_tt = bqplot.Tooltip(fields = ['name','color'], labels = ['state','duration_seconds'])
states_map = bqplot.Map(map_data=bqplot.topo_load('map_data/USStatesMap.json'),
                        interactions = {'click':'select','hover':'tooltip'},
                        unslected_styles = {'opacity': 0.4},
                        **map_styles, tooltip = map_tt, display_legend = False, labels=['UFO Sightings'])
fig_map = bqplot.Figure(marks=[states_map], fig_margins = {60,60,40,40},
                    title='UFO Sightings in the United States')

In [ ]:
def observe_selected(change):
    print(states_map.selected[-1])
    for abbr,fips in abbr_to_fits.items():
        if fips == str(states_map.selected[-1]):
            state_selected = abbr.lower()
            if cap_select.value == 'Total Sightings':
                plot_all.x = total_sightings[state_selected].index.values
                plot_all.y = total_sightings[state_selected][:]
    print(states_map.selected)
states_map.observe(observe_selected, 'selected')

In [ ]:
plot_all = Lines(x=total_sightings_year.index.values,y=total_sightings_year[:],colors = ['orange'],scales={'x':dt_x_fast,'y':lin_y})
fig_all = Figure(marks=[plot_all],axes = [x_ax, y_ay_S],)

In [ ]:
total_sightings_year.index.values

In [ ]:
HBox([VBox([cap_select,fig_map]),fig_all])

In [ ]:
len(ufo)